In [1]:
# !pip install undetected-chromedriver
from bs4 import BeautifulSoup
import requests
from datetime import datetime
import time
import re
import undetected_chromedriver as uc
import pandas as pd
import clipboard as c

In [3]:
def fetch_profile_links(driver, listing_url, scroll_times, save=True):
    links = []
    driver.get(listing_url)
    profile_class = "styles__StyledLink-sc-l6elh8-0 ikuMIO Blockreact__Block-sc-1xf18x6-0 kdnPIp AccountLink--ellipsis-overflow"
    for i in range(scroll_times):
        driver.execute_script("window.scrollBy(0, 1000)")
        page_source = driver.page_source
        time.sleep(3)
        soup = BeautifulSoup(page_source)
        user_tags = soup.find_all("a", {"class": profile_class})
        links.extend([f"{domain}/{i.get('href')}" for i in user_tags])
    links = list(set(links))
    if save:
        current_datetime = datetime.now().strftime('%Y%m%d%H%M%S')
        pd.DataFrame(links, columns=["Url"]).to_csv(f"Scrapped-Profiles-{current_datetime}.csv", index=False)
    return links


def fetch_profile_detail(driver, url, req_social):
    entities = []
    detail = {}
    driver.get(url)
    page_source = driver.page_source
    soup = BeautifulSoup(page_source)
    detail['profile_url'] = url
    detail['name'] = soup.find("div", {"class": "Overflowreact__OverflowContainer-sc-7qr9y8-0 jPSCbX AccountHeader--title"}).text
    info = soup.find_all("li", {"class": "Menureact__StyledListMenuItem-sc-1j0z9gq-3 hlufrI"})
    social_media_tag = soup.find_all("a", {"class": "styles__StyledLink-sc-l6elh8-0 ekTmzq Blockreact__Block-sc-1xf18x6-0 Buttonreact__StyledButton-sc-glfma3-0 kXZare kdWcfm ButtonGroupreact__StyledButton-sc-1skvztv-0 eztnHW AccountLinksBar--icon-button"})
    driver.find_element_by_xpath('//*[@id="main"]/div/div/div[1]/div[3]/div[3]/div/button').click() # To get address in clipboard
    time.sleep(1)
    detail["address"] = c.paste()
    
    if len(info) > 3:
        info = info[:3]
    for i in info:
        span = i.find_all('span')
        for s in span:
            entity = s.text
            if entity is not None:
                entities.append(entity)
        if len(span) < 2:
            entities.append("0")
    entities = dict(zip(entities[::2], entities[1::2]))
    detail.update(entities)
                    
    for tag in social_media_tag:
        url = tag.get("href")
        if url is not None:
            url_spl = re.split("/|.com", url)
            if len(url_spl) >= 2:
                domain = url_spl[2]
                if domain in req_social:
                    detail[domain] = url
                    
    return detail


def iterator(list_urls, driver, req_social, save=True, verbose=False):
    data = []
    try:
        for i, url in enumerate(list_urls):
            if verbose: print(i, "/", len(profile_links))
            detail = fetch_profile_detail(driver, url, req_social)
            data.append(detail)
    except:
        if verbose: print(f"Error at url {i} i-e, {url}")
    df = pd.DataFrame(data)
    if save:
        current_datetime = datetime.now().strftime('%Y%m%d%H%M%S')
        df.to_csv(f"Scrapped-Data-{current_datetime}.csv", index=False)
    return df
        

# Website detail

In [ ]:
domain = 'https://opensea.io'
listing_url = domain + "//" + "activity"

# Open a driver

In [5]:
driver = uc.Chrome(use_subprocess=True)

# Extract unique profiles

In [6]:
profile_links = fetch_profile_links(driver, listing_url, scroll_times=30, save=True)

# Extract profile data

In [7]:
req_social = ["twitter", "instagram"]
df = iterator(profile_links, driver, req_social, save=True, verbose=True)
df

0 / 61


C:\Users\hamza.usman\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  if sys.path[0] == '':


1 / 61
2 / 61
3 / 61
4 / 61
5 / 61
6 / 61
7 / 61
8 / 61
9 / 61
10 / 61
11 / 61
12 / 61
13 / 61
14 / 61
15 / 61
16 / 61
17 / 61
18 / 61
19 / 61
20 / 61
21 / 61
22 / 61
23 / 61
24 / 61
25 / 61
26 / 61
27 / 61
28 / 61
29 / 61
30 / 61
31 / 61
32 / 61
33 / 61
34 / 61
35 / 61
36 / 61
37 / 61
38 / 61
39 / 61
40 / 61
41 / 61
42 / 61
43 / 61
44 / 61
45 / 61
46 / 61
47 / 61
48 / 61
49 / 61
50 / 61
51 / 61
52 / 61
53 / 61
54 / 61
55 / 61
Error at url 55 i-e, https://opensea.io//creep6


,profile_url,name,address,address2,Collected,Created,Favorited,twitter,instagram
0,https://opensea.io//KJ_ETH,KJ_ETH,0x5347...7d39,0x5347f56a6DE5BF062Af973A27b41C5518EDe7d39,19,0,2,NaN,NaN
1,https://opensea.io//Oliverrr,Oliverrr,0xf266...FF9c,0xf266EC07BcED0c64514Dc0F7Ab1Bbc56Dc2AFF9c,503,0,15,NaN,NaN
2,https://opensea.io//0x2af4b707e1DCe8fC345f38Cf...,Unnamed,0x2af4...76d5,0x2af4b707e1DCe8fC345f38CfeeAA2421e54976d5,4,0,0,NaN,NaN
3,https://opensea.io//0xDD66C65f4aB47fD904850b93...,Unnamed,0xDD66...3Ec3,0xDD66C65f4aB47fD904850b9342236aDed3813Ec3,14,0,0,NaN,NaN
4,https://opensea.io//0x2461,0x2461,0xaeE2...59e0,0xaeE26290aa92811Cf1446cA963c404570D8759e0,1,0,15,NaN,NaN
5,https://opensea.io//Mrjosco_Vault,Mrjosco_Vault,0xA90E...DFb9,0xA90E6Ff084580B1194f7333a702De7641E33DFb9,20,0,2,https://twitter.com/@mrjosco,NaN
6,https://opensea.io//0x8DCeeb78462b002d71526ad0...,Unnamed,0x8DCe...1367,0x8DCeeb78462b002d71526ad0CeFB68bC3B001367,11,0,0,NaN,NaN
7,https://opensea.io//Wfg_pugs,Wfg_pugs,0xD744...68F1,0xD7448Fd62AD870C0ac16b6831d36078f3cf768F1,50,0,1.4K,NaN,https://instagram.com/wfg.pugs
8,https://opensea.io//schwags,schwags,0x679A...2444,0x679A5797828caa706F03879fd38dc5D3965D2444,12,0,1,https://twitter.com/@schwags_eth,NaN
9,https://opensea.io//KTuck28,KTuck28,0xe40c...62d0,0xe40c8deA5EdAB02C3B778605cf7b9dD1301062d0,39,0,28,NaN,NaN
